In [16]:
import pandas as pd
import numpy as np


In [17]:
# !ls /datasets/bikedata

### Data Setup
Pulling in all data files to begin our work

In [18]:
sept_2021 = pd.read_csv("/datasets/bikedata/202109-divvy-tripdata.csv")
oct_2021 = pd.read_csv("/datasets/bikedata/202110-divvy-tripdata.csv")
nov_2021 = pd.read_csv("/datasets/bikedata/202111-divvy-tripdata.csv")
dec_2021 = pd.read_csv("/datasets/bikedata/202112-divvy-tripdata.csv")
jan_2022 = pd.read_csv("/datasets/bikedata/202201-divvy-tripdata.csv")
feb_2022 = pd.read_csv("/datasets/bikedata/202202-divvy-tripdata.csv")
mar_2022 = pd.read_csv("/datasets/bikedata/202203-divvy-tripdata.csv")
apr_2022 = pd.read_csv("/datasets/bikedata/202204-divvy-tripdata.csv")
may_2022 = pd.read_csv("/datasets/bikedata/202205-divvy-tripdata.csv")
jun_2022 = pd.read_csv("/datasets/bikedata/202206-divvy-tripdata.csv")
jul_2022 = pd.read_csv("/datasets/bikedata/202207-divvy-tripdata.csv")
Aug_2022 = pd.read_csv("/datasets/bikedata/202208-divvy-tripdata.csv")


### Merging Data Frames
After confirming column names of each dataframe are identical, we merge them into a single df for bulk work.

In [19]:
merge_frames = [sept_2021,oct_2021,nov_2021,dec_2021,jan_2022,feb_2022,mar_2022,apr_2022,may_2022,jun_2022,jul_2022,Aug_2022]

full_cycle_data = pd.concat(merge_frames)
# Confirming that data merge worked
full_cycle_data.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,9DC7B962304CBFD8,electric_bike,2021-09-28 16:07:10,2021-09-28 16:09:54,NaN,NaN,NaN,NaN,41.89,-87.68,41.89,-87.67,casual
1,F930E2C6872D6B32,electric_bike,2021-09-28 14:24:51,2021-09-28 14:40:05,NaN,NaN,NaN,NaN,41.94,-87.64,41.98,-87.67,casual
2,6EF72137900BB910,electric_bike,2021-09-28 00:20:16,2021-09-28 00:23:57,NaN,NaN,NaN,NaN,41.81,-87.72,41.80,-87.72,casual
3,78D1DE133B3DBF55,electric_bike,2021-09-28 14:51:17,2021-09-28 15:00:06,NaN,NaN,NaN,NaN,41.80,-87.72,41.81,-87.72,casual
4,E03D4ACDCAEF6E00,electric_bike,2021-09-28 09:53:12,2021-09-28 10:03:44,NaN,NaN,NaN,NaN,41.88,-87.74,41.88,-87.71,casual


### Cleaning

In [20]:
# Checking Data types
full_cycle_data.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

- started_at & ended_at are objects
We need to translate to datetime so we can use them to calculate trip durations

In [21]:
full_cycle_data = full_cycle_data.drop_duplicates()
full_cycle_data['started_at'] = pd.to_datetime(full_cycle_data['started_at'])
full_cycle_data["ended_at"] = pd.to_datetime(full_cycle_data['ended_at'])


In [22]:
# changing column name to be more descriptive

full_cycle_data.rename(columns={"member_casual": "rider_type"}, inplace=True)


In [23]:
# Adding Weekday column for analysis
full_cycle_data["weekday"] = full_cycle_data['started_at'].dt.day_name()
#Adding Month Column for analysis
full_cycle_data["month"] = full_cycle_data['started_at'].dt.month_name()


In [24]:
full_cycle_data['trip_duration'] = full_cycle_data['ended_at'] - full_cycle_data['started_at']
full_cycle_data['trip_duration'] = round(full_cycle_data['trip_duration'].dt.seconds / 60,2)


In [25]:
full_cycle_data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,rider_type,weekday,month,trip_duration
0,9DC7B962304CBFD8,electric_bike,2021-09-28 16:07:10,2021-09-28 16:09:54,NaN,NaN,NaN,NaN,41.89,-87.68,41.89,-87.67,casual,Tuesday,September,2.73
1,F930E2C6872D6B32,electric_bike,2021-09-28 14:24:51,2021-09-28 14:40:05,NaN,NaN,NaN,NaN,41.94,-87.64,41.98,-87.67,casual,Tuesday,September,15.23
2,6EF72137900BB910,electric_bike,2021-09-28 00:20:16,2021-09-28 00:23:57,NaN,NaN,NaN,NaN,41.81,-87.72,41.80,-87.72,casual,Tuesday,September,3.68
3,78D1DE133B3DBF55,electric_bike,2021-09-28 14:51:17,2021-09-28 15:00:06,NaN,NaN,NaN,NaN,41.80,-87.72,41.81,-87.72,casual,Tuesday,September,8.82
4,E03D4ACDCAEF6E00,electric_bike,2021-09-28 09:53:12,2021-09-28 10:03:44,NaN,NaN,NaN,NaN,41.88,-87.74,41.88,-87.71,casual,Tuesday,September,10.53


Ran through some quick queries to check for NULL values on key pieces of data. 
- If start or end time or location was missing, that entry would have to be removed. 
```
SELECT count(*) FROM full_cycle_data WHERE start_at IS NULL
SELECT count(*) FROM full_cycle_data WHERE end_at IS NULL
SELECT count(*) FROM full_cycle_data WHERE start_lat IS NULL
SELECT count(*) FROM full_cycle_data WHERE end_lat IS NULL
```

### Cleaning inputs by trip length
After running the query below we discovered 110578 trips with a duration below 60s. 
Conducting some research on Divvy's public data, they state trips under 60s should be prunded, as they likely attribute to false starts. 
```
SELECT count(*) FROM full_cycle_data WHERE trip_duration < 1
```

In [26]:
len(full_cycle_data)

5883043

In [27]:
full_cycle_data.describe()

,start_lat,start_lng,end_lat,end_lng,trip_duration
count,5.883043e+06,5.883043e+06,5.877316e+06,5.877316e+06,5.883043e+06
mean,4.190104e+01,-8.764766e+01,4.190129e+01,-8.764786e+01,1.722305e+01
std,4.719795e-02,3.097188e-02,4.729980e-02,3.059219e-02,3.379055e+01
min,4.164000e+01,-8.784000e+01,4.139000e+01,-8.897000e+01,0.000000e+00
25%,4.188103e+01,-8.766201e+01,4.188103e+01,-8.766356e+01,6.050000e+00
50%,4.189993e+01,-8.764377e+01,4.190000e+01,-8.764410e+01,1.072000e+01
75%,4.193000e+01,-8.762930e+01,4.193000e+01,-8.762932e+01,1.933000e+01
max,4.563503e+01,-7.379648e+01,4.237000e+01,-8.750000e+01,1.439980e+03


In [32]:
full_cycle_data[full_cycle_data.trip_duration < 1].head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,rider_type,weekday,month,trip_duration
13,3575930BD49A07EB,electric_bike,2021-09-29 14:06:59,2021-09-29 14:07:25,NaN,NaN,NaN,NaN,41.94,-87.71,41.94,-87.71,casual,Wednesday,September,0.43
50,8A306A960FE6D900,electric_bike,2021-09-08 13:23:42,2021-09-08 13:24:19,NaN,NaN,NaN,NaN,41.92,-87.66,41.92,-87.66,casual,Wednesday,September,0.62
65,3829276BFF6724F9,electric_bike,2021-09-05 16:07:51,2021-09-05 16:08:16,NaN,NaN,NaN,NaN,41.96,-87.64,41.96,-87.64,casual,Sunday,September,0.42
127,891428E27EA74D11,electric_bike,2021-09-27 14:18:45,2021-09-27 14:19:14,NaN,NaN,NaN,NaN,41.88,-87.64,41.88,-87.64,casual,Monday,September,0.48
170,3C0F3AF98F5D74D7,electric_bike,2021-09-08 17:36:55,2021-09-08 17:37:08,NaN,NaN,NaN,NaN,41.90,-87.62,41.90,-87.62,casual,Wednesday,September,0.22


In [ ]:
# We can now see or minimum trip duration is 1 min. 
full_cycle_data.drop(full_cycle_data[full_cycle_data['trip_duration'] < 1].index, inplace=True)
full_cycle_data.describe()

,start_lat,start_lng,end_lat,end_lng,trip_duration
count,5.048467e+06,5.048467e+06,5.043362e+06,5.043362e+06,5.048467e+06
mean,4.190110e+01,-8.764762e+01,4.190136e+01,-8.764781e+01,1.757932e+01
std,4.714167e-02,3.041025e-02,4.726945e-02,3.056521e-02,3.404296e+01
min,4.164000e+01,-8.784000e+01,4.139000e+01,-8.897000e+01,1.000000e+00
25%,4.188103e+01,-8.766201e+01,4.188103e+01,-8.766338e+01,6.280000e+00
50%,4.190000e+01,-8.764375e+01,4.190000e+01,-8.764410e+01,1.097000e+01
75%,4.193000e+01,-8.762928e+01,4.193000e+01,-8.762932e+01,1.963000e+01
max,4.207000e+01,-8.752000e+01,4.237000e+01,-8.750000e+01,1.439980e+03


### Exporting clean tables for analysis
- Wanted to break into two smaller tables, one for customer analysis, where station location was not needed. 
- One to draw some insights out of bike location use. 

In [36]:
trips_clean = full_cycle_data.drop(['start_station_name','start_station_id','end_station_name','end_station_id', 'start_lat', 'start_lng','end_lat', 'end_lng'], axis=1)
trips_clean.head()
trips_clean.to_csv('trips_clean.csv',index=False)

In [37]:
locations_clean = full_cycle_data.drop(['rideable_type','month', 'trip_duration'], axis=1)
locations_clean.head()
locations_clean.to_csv('locations_clean.csv',index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=adbe9eb0-06fc-406e-a320-733a903fe456' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>